In [1]:
import pandas as pd

df = pd.read_csv("Creditcard_data.csv")
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
df['Class'].value_counts()


,count
Class,
0,763
1,9


In [3]:
X = df.drop(columns=['Class'])
y = df['Class']


In [4]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_bal, y_bal = smote.fit_resample(X, y)

print(y_bal.value_counts())


Class
0    763
1    763
Name: count, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split

samples = []

for i in range(5):
    X_train, _, y_train, _ = train_test_split(
        X_bal, y_bal, test_size=0.3, random_state=i
    )
    samples.append((X_train, y_train))


In [6]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN, SMOTETomek

sampling_methods = [
    RandomOverSampler(),
    RandomUnderSampler(),
    SMOTE(),
    SMOTEENN(),
    SMOTETomek()
]


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

models = [
    LogisticRegression(max_iter=1000),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(),
    GaussianNB()
]


In [8]:
from sklearn.metrics import accuracy_score

results = []

for i, (X_s, y_s) in enumerate(samples):
    for j, sampler in enumerate(sampling_methods):
        X_res, y_res = sampler.fit_resample(X_s, y_s)

        for k, model in enumerate(models):
            model.fit(X_res, y_res)
            y_pred = model.predict(X_res)
            acc = accuracy_score(y_res, y_pred)

            results.append([f"M{k+1}", f"Sampling{j+1}", acc])


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [9]:
results_df = pd.DataFrame(
    results, columns=["Model", "Sampling Technique", "Accuracy"]
)

results_df


,Model,Sampling Technique,Accuracy
0,M1,Sampling1,0.928439
1,M2,Sampling1,1.000000
2,M3,Sampling1,1.000000
3,M4,Sampling1,0.688662
4,M5,Sampling1,0.868959
...,...,...,...
120,M1,Sampling5,0.931992
121,M2,Sampling5,1.000000
122,M3,Sampling5,1.000000
123,M4,Sampling5,0.674330


In [10]:
best_results = results_df.loc[
    results_df.groupby("Model")["Accuracy"].idxmax()
]

best_results


,Model,Sampling Technique,Accuracy
115,M1,Sampling4,0.975703
1,M2,Sampling1,1.000000
2,M3,Sampling1,1.000000
93,M4,Sampling4,0.797954
69,M5,Sampling4,0.893179


In [11]:
results_df.to_csv("sampling_results.csv", index=False)
